In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn import tree
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Research/Fetal Health/fetal_health.csv')
X = df.drop(columns=['fetal_health'])
y = df['fetal_health']

In [ ]:
# XGBoost

X = df.drop(columns=['fetal_health'], axis=1)
y = df['fetal_health']-1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(objective='multi:softmax', num_class=3)

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [100, 500, 1000],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0]
}

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98       333
         1.0       0.92      0.84      0.88        64
         2.0       0.93      0.97      0.95        29

    accuracy                           0.96       426
   macro avg       0.94      0.93      0.93       426
weighted avg       0.96      0.96      0.96       426



In [ ]:
# Random Forest

X = df.drop(columns=['fetal_health'])
y = df['fetal_health']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10],
}

grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         1.0       0.96      0.98      0.97       332
         2.0       0.86      0.76      0.81        55
         3.0       0.97      0.87      0.92        39

    accuracy                           0.95       426
   macro avg       0.93      0.87      0.90       426
weighted avg       0.94      0.95      0.94       426



In [ ]:
# LightGBM

df = pd.read_csv('/content/drive/My Drive/Research/Fetal Health/fetal_health.csv')
df_light = df.drop_duplicates()
X = df_light.drop(columns=['fetal_health'])
y = df_light['fetal_health']-1

X.columns = [col.replace(' ', '_') for col in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'objective': ['multiclass'],
    'num_classes': [3],
    'metric': ['multi_error'],
    'boosting_type': ['gbdt'],
    'num_leaves': [31, 50],
    'learning_rate': [0.05, 0.1],
    'force_col_wise': [True]
}

clf = lgb.LGBMClassifier()

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
model = lgb.LGBMClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

report = classification_report(y_test, y_pred)
print()
print(report)


              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98       341
         1.0       0.92      0.85      0.88        54
         2.0       1.00      1.00      1.00        28

    accuracy                           0.97       423
   macro avg       0.97      0.95      0.96       423
weighted avg       0.97      0.97      0.97       423

